In [46]:
import pandas as pd
from pathlib import Path
import en_core_web_md as spacy_model
from tqdm import tqdm
import json

In [47]:
tweets = pd.read_csv(Path('../data/tweets_11-06-2020.csv'))
tweets.head()


,id,text,isRetweet,isDeleted,device,favorites,retweets,date
0,98454970654916608,Republicans and Democrats have both created ou...,f,f,TweetDeck,49,255,2011-08-02 18:07:48
1,1234653427789070336,I was thrilled to be back in the Great city of...,f,f,Twitter for iPhone,73748,17404,2020-03-03 01:34:50
2,1218010753434820614,RT @CBS_Herridge: READ: Letter to surveillance...,t,f,Twitter for iPhone,0,7396,2020-01-17 03:22:47
3,1304875170860015617,The Unsolicited Mail In Ballot Scam is a major...,f,f,Twitter for iPhone,80527,23502,2020-09-12 20:10:58
4,1218159531554897920,RT @MZHemingway: Very friendly telling of even...,t,f,Twitter for iPhone,0,9081,2020-01-17 13:13:59


In [48]:
nlp = spacy_model.load()

In [49]:
# Lets try it out on the first record
text = tweets.loc[0,'text']
doc = nlp(text)
print(text)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Republicans and Democrats have both created our economic problems.
Republicans 0 11 NORP
Democrats 16 25 NORP


In [50]:
texts = tweets['text']
docs = [doc for doc in nlp.pipe(tqdm(texts), disable=["tagger", "parser"])]

100%|██████████| 55090/55090 [00:58<00:00, 942.48it/s] 


In [51]:
def doc_to_ner(doc):
    result = {}
    result['entities'] = [{'text': ent.text, 'start_char': ent.start_char, 'end_char': ent.end_char, 'label': ent.label_} for ent in doc.ents]
    return result
print(doc_to_ner(docs[0]))

{'entities': [{'text': 'Republicans', 'start_char': 0, 'end_char': 11, 'label': 'NORP'}, {'text': 'Democrats', 'start_char': 16, 'end_char': 25, 'label': 'NORP'}]}


In [52]:
print(json.dumps([doc_to_ner(doc) for doc in docs][1:100]))

[{"entities": [{"text": "Charlotte", "start_char": 47, "end_char": 56, "label": "GPE"}, {"text": "North Carolina", "start_char": 58, "end_char": 72, "label": "GPE"}, {"text": "thousands", "start_char": 78, "end_char": 87, "label": "CARDINAL"}, {"text": "American", "start_char": 103, "end_char": 111, "label": "NORP"}, {"text": "AMERICA", "start_char": 196, "end_char": 203, "label": "GPE"}, {"text": "FIRST", "start_char": 204, "end_char": 209, "label": "ORDINAL"}, {"text": "a wonderful evening", "start_char": 225, "end_char": 244, "label": "TIME"}, {"text": "#KAG2020", "start_char": 247, "end_char": 255, "label": "PERSON"}]}, {"entities": [{"text": "CBS News", "start_char": 65, "end_char": 73, "label": "ORG"}]}, {"entities": [{"text": "The Unsolicited Mail In Ballot Scam", "start_char": 0, "end_char": 35, "label": "ORG"}, {"text": "Democracy, &amp", "start_char": 61, "end_char": 76, "label": "ORG"}, {"text": "Democrats", "start_char": 82, "end_char": 91, "label": "NORP"}, {"text": "Ballo